<a href="https://colab.research.google.com/github/PhilNguyenN/KernelizedFilterTracking/blob/master/%08FastTracking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import files
from google.colab.patches import cv2_imshow
import zipfile
import io
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
from numpy import dot
from numpy.linalg import norm
import scipy.fft
from scipy.linalg import dft
from scipy.ndimage.interpolation import shift
import zipfile
from scipy import misc, fftpack
import math
from skimage.feature import hog
from skimage import data, exposure
!pip install ipython-autotime
%load_ext autotime

In [0]:
data = zipfile.ZipFile("/content/Girl.zip", 'r')
data.extractall()
data = zipfile.ZipFile("/content/FaceOcc2.zip", 'r')
data.extractall()
# data = zipfile.ZipFile("/content/Freeman4.zip", 'r')
# data.extractall()
# data = zipfile.ZipFile("/content/Liquor.zip", 'r')
# data.extractall()
data = zipfile.ZipFile("/content/Dog1.zip", 'r')
data.extractall()

In [3]:
def videoInfor(video_path):
  if video_path[-1] != '/':
    video_path = video_path + '/'

  gt_path = video_path + "groundtruth_rect.txt"

  try:
    ground_truth = np.loadtxt(gt_path,dtype=[('f0',int),('f1',int),('f2',int),('f3',int)])
  except:
    ground_truth = np.loadtxt(gt_path,delimiter=',',dtype=[('f0',int),('f1',int),('f2',int),('f3',int)])


  #Initial position and targer size 
  target_sz = np.array([ground_truth[0][3],ground_truth[0][2]]) # HxW
  #Center position
  pos = np.array([ground_truth[0][1],ground_truth[0][0]]) + np.floor(target_sz/2)

  temp = np.zeros((ground_truth.shape[0],2))

  # #Store positions instead of boxes
  # for i in range(ground_truth.shape[0]):
  #   temp[i][0] = ground_truth[i][1] + ground_truth[i][3]/2 # y
  #   temp[i][1] = ground_truth[i][0] + ground_truth[i][2]/2 # x
  # ground_truth = temp

  img_path = video_path + "img"
  pictureSequences = os.listdir(img_path)

  img_files = np.array([])

  for i in sorted(pictureSequences):
    img_files = np.append(img_files,i)

  return (img_files,pos,target_sz,ground_truth,video_path)


def gaussian_shaped_labels(sigma,sz):
  #CHECKED
  cs,rs = np.meshgrid(np.arange(0,sz[1])-np.floor(sz[1]/2),np.arange(0,sz[0]) - np.floor(sz[0]/2))
  cs = cs + 1
  rs = rs + 1
  labels = np.exp(-0.5*(rs**2 + cs**2)/sigma**2)
  labels = np.roll(labels, int(-np.floor(sz[0]/2)+1),axis=0)
  labels = np.roll(labels, int(-np.floor(sz[1]/2)+1),axis=1)
  # plt.imshow(labels)
  # print(labels)
  if np.round(labels[0][0]) !=1:
    print('LOI CMNR')
  return labels

def smallerCont_loop(a,conditionValue,assignedValue):
  # CHECKED
  for i,num in enumerate(a):
    if num < conditionValue:
      a[i] = assignedValue
  return a

def largerCont_loop(a,conditionValue,assignedValue):
  # CHECKED
  for i,num in enumerate(a):
    if num > conditionValue:
      a[i] = assignedValue
  return a

def get_features(im,cos_window):
  # x = hog(im,orientations=9,pixels_per_cell=(4,4),cells_per_block=(2,2))
  x = np.double(im)/255
  x = x - np.mean(x[:])  
  if cos_window.size != 0:
    x = x*cos_window
	
  return x

def gaussian_correlation(xf,yf,sigma):
  N = xf.shape[0]*xf.shape[1]
  
  xx = np.linalg.norm(xf.ravel(), ord=2)**2 /N
  yy = np.linalg.norm(yf.ravel(), ord=2)**2 /N
  
  xyf = xf*np.conjugate(yf)
  xy = np.real(scipy.fft.ifft2(xyf))
  # print(xy)
  sss = ((xx+yy -2*xy)/N)
  # print(sss)
  Kf = scipy.fft.fft2(np.exp(-1*sss/sigma**2))
  
  return Kf

def get_subwindow(im,pos,sz):
  # unCHECKED
  if np.isscalar(sz):
    sz = np.array([sz, sz])
    #pos[43,72], wsz[112,77]
 
  ys = np.floor(pos[0]) + np.arange(0,sz[0]) - np.floor((sz[0])/2) +1
  xs = np.floor(pos[1]) + np.arange(0,sz[1]) - np.floor((sz[1])/2) +1 
  
  xs = smallerCont_loop(xs,1,1)
  ys = smallerCont_loop(ys,1,1)
  xs = largerCont_loop(xs,im.shape[1],im.shape[1])
  ys = largerCont_loop(ys,im.shape[0],im.shape[0])

  # print(ys)
  # print(xs)

  out = np.zeros((ys.shape[0],xs.shape[0]))
 
  for i,data_y in enumerate(ys):
    for j,data_x in enumerate(xs):
      temp =  im[int(data_y)-1][int(data_x)-1]
      # print(i,j,data_y,data_x)
      out[i][j] = temp
  # print(out.shape)
  ''' TEST 
    im = cv2.imread('/content/Girl/img/0001.jpg')
    im = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
    resize_image = (np.sqrt(np.prod(target_sz)) >= 100)

    if resize_image:
    pos = np.floor(pos/2)
    target_sz = np.floor(target_sz/2);

    window_sz = np.floor(target_sz * (1 + padding))
    patch = get_subwindow(im,pos,window_sz)
    print(patch)
    cv2_imshow(patch)
  '''
  return out

# im = cv2.imread('/content/Girl/img/0001.jpg')
# im = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
# resize_image = (np.sqrt(np.prod(target_sz)) >= 100)

# if resize_image:
#   pos = np.floor(pos/2)
#   target_sz = np.floor(target_sz/2);

# window_sz = np.floor(target_sz * (1 + padding))
# patch = get_subwindow(im,pos,window_sz)
# # print(patch)
# cv2_imshow(patch)

time: 165 ms


In [19]:
def tracker(video_path,img_files,pos,target_sz,padding,
            kernel,lambda_reg,output_sigma_factor,interp_factor,
            cell_size,features,gt):

  resize_image = (np.sqrt(np.prod(target_sz)) >= 100)
  if resize_image:
    pos = np.floor(pos/2)
    target_sz = np.floor(target_sz/2);
    for i in range(gt.shape[0]):
      for j in range(4):
        gt[i][j] = gt[i][j]/2
    

  def resize_image_true(resize_image):
    if resize_image:
      sample = cv2.imread(video_path + "img/0001.jpg")
      sample = cv2.resize(sample,(int(sample.shape[1]/2),int(sample.shape[0]/2)))
      width,heigh,_ = sample.shape
    else:
      sample = cv2.imread(video_path + "img/0001.jpg")
      width,heigh,_ = sample.shape
    return width,heigh
  # Recorded video
  width, heigh = resize_image_true(resize_image)
  # print(width,heigh)
  res=(heigh,width) #video resolution
  fourcc = cv2.VideoWriter_fourcc(*'MP4V') #codec
  video = cv2.VideoWriter('Tracking.mp4', fourcc, 20.0, res)

  # window size, taking padding into account
  window_sz = np.floor(target_sz*(1+padding))
  
  # create regression labels, gaussian shaped, with a bandwidth
	# proportional to target size
  output_sigma = np.sqrt(np.prod(target_sz)) * output_sigma_factor/cell_size
  yf = scipy.fft.fft2(gaussian_shaped_labels(output_sigma, np.floor(window_sz / cell_size)))
  
  # store pre-computed cosine window 
  # CHECKED
  cos_window = np.hanning(yf.shape[0]).reshape(yf.shape[0],1)*np.hanning(yf.shape[1]).reshape(1,yf.shape[1])
  
  ''' Test cos_window 
    output_sigma = 0.9337
    window_sz = np.array([112,77])
    cell_size = 4
    yf = scipy.fft.fft2(gaussian_shaped_labels(output_sigma, np.floor(window_sz / cell_size)))
    cos_window = np.hanning(yf.shape[0]).reshape(yf.shape[0],1)*np.hanning(yf.shape[1]).reshape(1,yf.shape[1])
  '''

  positions = np.zeros((img_files.shape[0],2)) #to calculate precision
  
  for i,frame in enumerate(img_files):
    #Load images
    im = cv2.imread(video_path + "img/" + frame)

    if im.shape[2] > 1:
      im = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
    if resize_image:
      im = cv2.resize(im,(int(im.shape[1]/2),int(im.shape[0]/2)))

    if i > 0:
      #Get the subwindow for detection at the postion from the last frame 
      #and convert to Fdomain 
      patch = get_subwindow(im, pos, window_sz)
      
      zf = scipy.fft.fft2(get_features(patch,cos_window))

      kzf = gaussian_correlation(zf,model_xf,kernel.sigma)

      response = np.real(scipy.fft.ifft2(np.multiply(model_alphaf,kzf)))

      temp = np.argwhere(response == (np.max(response[:]))) # NEED FIXed
      # if i == 10:
      #   ssss
      vert_delta = temp[0][0]
      horiz_delta = temp[0][1]
      
      if vert_delta > zf.shape[0]/2:
        vert_delta = vert_delta - zf.shape[0]
      if horiz_delta > zf.shape[1]/2:
      	horiz_delta = horiz_delta - zf.shape[1]

      pos = pos + cell_size*np.array([vert_delta,horiz_delta])
      
  
    patch = get_subwindow(im, pos, window_sz)
  
    xf = scipy.fft.fft2(get_features(patch,cos_window))
    kf = gaussian_correlation(xf, xf, kernel.sigma)

    alphaf = np.divide(yf,(kf+lambda_reg))

    if i == 0:
      model_alphaf = alphaf
      model_xf = xf
    else:
      model_alphaf = (1-interp_factor)*model_alphaf + interp_factor*alphaf
      model_xf = (1-interp_factor)*model_xf + interp_factor*xf
    # print(i)
    # print(model_alphaf)
    positions[i,:] = pos
    
    t_left  = pos[::-1]- np.round(target_sz[::-1]/2)
    b_right = pos[::-1]+ np.round(target_sz[::-1]/2)
    
    im = cv2.imread(video_path + "img/" + frame)
    if resize_image:
      im = cv2.resize(im,(int(im.shape[1]/2),int(im.shape[0]/2)))

    boxedImage = cv2.rectangle(im,(int(t_left[0]),int(t_left[1])),(int(b_right[0]),int(b_right[1])),(0,255,0),1)
    boxedImage = cv2.putText(boxedImage,'Fast Tracking',(int(t_left[0]),int(t_left[1]) - 5), cv2.FONT_HERSHEY_SIMPLEX , 0.2, (36,255,12), 1)
    boxedImage = cv2.rectangle(boxedImage,(int(gt[i][0]),int(gt[i][1])),(int(gt[i][0]) + int(gt[i][2]),int(gt[i][1])+int(gt[i][3])),(0,255,255),1)
    boxedImage = cv2.putText(boxedImage,'Ground Truth',(int(gt[i][0]),int(gt[i][1]) - 2), cv2.FONT_HERSHEY_SIMPLEX , 0.2, (0,255,122), 1)
    # cv2_imshow(boxedImage)
    video.write(boxedImage)
    if i == gt.size-1:
      break

  # video.release()
  if resize_image:
    positions = positions*2



time: 162 ms


In [12]:
#Initial Parameters 
interp_factor = 0.075
cell_size = 1
padding = 1.5  
lambda_reg = 1e-4  
output_sigma_factor = 0.1;

class kernel_struct: 
  #Class kernel
  def __init__(self,sigma ,poly_a ,poly_b,kernel_type):
    self.sigma = sigma
    self.poly_a = poly_a
    self.poly_b = poly_b
    self.kernel_type = kernel_type
    
class features_struct:
  def __init__(self,features_type,hog,hog_orientations):
    self.features_type = features_type
    self.hog = hog
    self.hog_orientations = hog_orientations

kernel = kernel_struct(0.2,1,7,'gaussian')
features = features_struct('gray',False,9)

video_path = '/content/FaceOcc2/'
img_files, pos, target_sz, ground_truth, video_path =  videoInfor(video_path)


time: 35.6 ms


In [21]:
tracker(video_path,img_files,pos,target_sz,padding,kernel,lambda_reg,output_sigma_factor,
        interp_factor,cell_size,features,ground_truth)

time: 3min 13s
